In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
#Importation de la base de donnees
data = pd.read_csv('C:/Users/win/Documents/M2 SEP/S1/Cousin/Donnees_equilibrees.csv',sep = ",")
stat = pd.read_csv('C:/Users/win/Documents/M2 SEP/Base de données maritimes modif/operations_stats.csv', sep=";",encoding = 'utf-8')
ope = pd.read_csv('C:/Users/win/Documents/M2 SEP/Base de données maritimes modif/operations.csv',sep = ";",encoding = 'utf-8')
flot = pd.read_csv('C:/Users/win/Documents/M2 SEP/Base de données maritimes modif/flotteurs_utf.csv',sep = ";",)
res_humain = pd.read_csv('C:/Users/win/Documents/M2 SEP/Base de données maritimes modif/resultats_humain.csv',sep = ";",encoding = 'utf-8')



C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3165: DtypeWarning: Columns (0,5,9,11) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [3]:
join1 = pd.merge(stat,ope,on = 'operation_id',how='inner')
join2 = pd.merge(flot,join1,on = 'operation_id',how='inner')
data = pd.merge(res_humain,join2,on = 'operation_id',how='inner')

# Visualisation des données


In [4]:
test = data.copy()


In [5]:
#Ajout de la partie d'Alizée
test = test.dropna(axis=0)
test.shape


(55359, 24)

In [6]:

#Valeur Target
data['mort'].value_counts()


0    193592
1      5826
Name: mort, dtype: int64

In [7]:
#Convertir la variable "est_vacances_scolaires" en booléen
test["est_vacances_scolaires"] = test["est_vacances_scolaires"].astype("bool")
test.dtypes

operation_id                                 object
categorie_personne                           object
type_flotteur                                object
categorie_flotteur                           object
mois                                          int64
mois_texte                                   object
est_weekend                                    bool
est_jour_ferie                                 bool
est_vacances_scolaires                         bool
phase_journee                                object
distance_cote_metres                        float64
distance_cote_milles_nautiques              float64
maree_port                                   object
maree_coefficient                           float64
maree_categorie                              object
nombre_personnes_assistees                    int64
nombre_personnes_tous_deces                   int64
nombre_personnes_tous_deces_ou_disparues      int64
mort                                          int64
nombre_perso

In [8]:
flottant = list(test.select_dtypes('float').columns)
flottant
#On selectionne les flottants

['distance_cote_metres',
 'distance_cote_milles_nautiques',
 'maree_coefficient',
 'vent_force',
 'mer_force']

In [9]:
objet = list(test.select_dtypes('object').columns)

objet
#On selectionne les objets

['operation_id',
 'categorie_personne',
 'type_flotteur',
 'categorie_flotteur',
 'mois_texte',
 'phase_journee',
 'maree_port',
 'maree_categorie',
 'departement',
 'zone_responsabilite']

In [10]:
booleen =list(test.select_dtypes('bool').columns)
booleen

#On selectionne les booleens

['est_weekend', 'est_jour_ferie', 'est_vacances_scolaires']

In [11]:
key_columns= ['mois','mort','departement','categorie_personne',
 'categorie_flotteur',
 'phase_journee',
 'maree_port']

#On selectionne les valeurs quali qu'on veut

In [12]:
df_quali = test[key_columns + booleen ]
df_quali

#On créé df composé des booleens et des quali

,mois,mort,departement,categorie_personne,categorie_flotteur,phase_journee,maree_port,est_weekend,est_jour_ferie,est_vacances_scolaires
2284,2,0,Somme,Plaisancier francais,Plaisance,apres-midi,Le Treport,True,False,True
2326,2,0,Nord,Pecheur francais,Peche,nuit,Dunkerque,False,False,True
2347,2,0,Somme,Autre,Loisir nautique,apres-midi,Le Touquet,True,False,True
2356,2,0,Pas-de-Calais,Clandestin,Plaisance,nuit,Wissant,False,False,False
2357,2,0,Pas-de-Calais,Clandestin,Plaisance,nuit,Wissant,False,False,False
...,...,...,...,...,...,...,...,...,...,...
190201,8,0,Gard,Toutes categories,Plaisance,apres-midi,Arcachon Eyrac,False,False,True
190202,8,0,Herault,Toutes categories,Plaisance,nuit,Arcachon Eyrac,False,False,True
190203,8,0,Herault,Toutes categories,Peche,apres-midi,Vieux-Boucau,True,True,True
190204,11,0,Aude,Toutes categories,Loisir nautique,apres-midi,Vieux-Boucau,True,False,False


In [13]:
df_quali = df_quali.reset_index(drop=True)
df_quali

#On reset les index 


,mois,mort,departement,categorie_personne,categorie_flotteur,phase_journee,maree_port,est_weekend,est_jour_ferie,est_vacances_scolaires
0,2,0,Somme,Plaisancier francais,Plaisance,apres-midi,Le Treport,True,False,True
1,2,0,Nord,Pecheur francais,Peche,nuit,Dunkerque,False,False,True
2,2,0,Somme,Autre,Loisir nautique,apres-midi,Le Touquet,True,False,True
3,2,0,Pas-de-Calais,Clandestin,Plaisance,nuit,Wissant,False,False,False
4,2,0,Pas-de-Calais,Clandestin,Plaisance,nuit,Wissant,False,False,False
...,...,...,...,...,...,...,...,...,...,...
55354,8,0,Gard,Toutes categories,Plaisance,apres-midi,Arcachon Eyrac,False,False,True
55355,8,0,Herault,Toutes categories,Plaisance,nuit,Arcachon Eyrac,False,False,True
55356,8,0,Herault,Toutes categories,Peche,apres-midi,Vieux-Boucau,True,True,True
55357,11,0,Aude,Toutes categories,Loisir nautique,apres-midi,Vieux-Boucau,True,False,False


In [14]:

from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

convert = test[flottant]



#On selectionne les variables quanti pour les centrées réduire obliger pour le SMOTE 

In [15]:

df_scaled = scaler.fit_transform(convert.to_numpy())
df_scaled



array([[-0.54311675, -0.54306593,  1.08764217, -0.34275064, -0.70052844],
       [ 3.98660057,  3.98649854, -0.69133547,  1.60293565,  1.42647301],
       [-0.54311675, -0.54306593, -0.93841569,  0.30581146,  0.36297229],
       ...,
       [ 0.56989951,  0.56895413, -0.59250338, -0.34275064, -1.76402917],
       [-0.31015985, -0.30819963, -0.93841569, -1.63987483,  0.36297229],
       [ 0.7252041 ,  0.7271294 ,  0.34640149, -1.63987483, -1.76402917]])

In [16]:
df_scaled = pd.DataFrame(df_scaled, columns = flottant)

df_scaled



,distance_cote_metres,distance_cote_milles_nautiques,maree_coefficient,vent_force,mer_force
0,-0.543117,-0.543066,1.087642,-0.342751,-0.700528
1,3.986601,3.986499,-0.691335,1.602936,1.426473
2,-0.543117,-0.543066,-0.938416,0.305811,0.362972
3,3.650107,3.650975,-0.493671,-0.342751,0.362972
4,3.650107,3.650975,-0.493671,-0.342751,0.362972
...,...,...,...,...,...
55354,-0.517233,-0.519100,0.000489,0.305811,0.362972
55355,-0.154855,-0.154818,0.000489,-0.342751,0.362972
55356,0.569900,0.568954,-0.592503,-0.342751,-1.764029
55357,-0.310160,-0.308200,-0.938416,-1.639875,0.362972


In [17]:
test = pd.DataFrame(scaler.inverse_transform(df_scaled),columns=flottant)
test

,distance_cote_metres,distance_cote_milles_nautiques,maree_coefficient,vent_force,mer_force
0,0.0,2.220446e-16,92.0,3.0,2.0
1,17500.0,9.450000e+00,56.0,6.0,4.0
2,0.0,2.220446e-16,51.0,4.0,3.0
3,16200.0,8.750000e+00,60.0,3.0,3.0
4,16200.0,8.750000e+00,60.0,3.0,3.0
...,...,...,...,...,...
55354,100.0,5.000000e-02,70.0,4.0,3.0
55355,1500.0,8.100000e-01,70.0,3.0,3.0
55356,4300.0,2.320000e+00,58.0,3.0,1.0
55357,900.0,4.900000e-01,51.0,1.0,3.0


In [18]:
df2 = pd.concat([df_quali,df_scaled],axis = 1,)
df2

#On créé df2 qui est notre dataframe centré réduit : il est bon

,mois,mort,departement,categorie_personne,categorie_flotteur,phase_journee,maree_port,est_weekend,est_jour_ferie,est_vacances_scolaires,distance_cote_metres,distance_cote_milles_nautiques,maree_coefficient,vent_force,mer_force
0,2,0,Somme,Plaisancier francais,Plaisance,apres-midi,Le Treport,True,False,True,-0.543117,-0.543066,1.087642,-0.342751,-0.700528
1,2,0,Nord,Pecheur francais,Peche,nuit,Dunkerque,False,False,True,3.986601,3.986499,-0.691335,1.602936,1.426473
2,2,0,Somme,Autre,Loisir nautique,apres-midi,Le Touquet,True,False,True,-0.543117,-0.543066,-0.938416,0.305811,0.362972
3,2,0,Pas-de-Calais,Clandestin,Plaisance,nuit,Wissant,False,False,False,3.650107,3.650975,-0.493671,-0.342751,0.362972
4,2,0,Pas-de-Calais,Clandestin,Plaisance,nuit,Wissant,False,False,False,3.650107,3.650975,-0.493671,-0.342751,0.362972
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55354,8,0,Gard,Toutes categories,Plaisance,apres-midi,Arcachon Eyrac,False,False,True,-0.517233,-0.519100,0.000489,0.305811,0.362972
55355,8,0,Herault,Toutes categories,Plaisance,nuit,Arcachon Eyrac,False,False,True,-0.154855,-0.154818,0.000489,-0.342751,0.362972
55356,8,0,Herault,Toutes categories,Peche,apres-midi,Vieux-Boucau,True,True,True,0.569900,0.568954,-0.592503,-0.342751,-1.764029
55357,11,0,Aude,Toutes categories,Loisir nautique,apres-midi,Vieux-Boucau,True,False,False,-0.310160,-0.308200,-0.938416,-1.639875,0.362972


In [19]:
df2['mort'].value_counts(normalize = True)
#On vérifie si la variable Mort est toujours ok

0    0.977203
1    0.022797
Name: mort, dtype: float64

In [20]:
df =df2
df
#On met df2 dans df

,mois,mort,departement,categorie_personne,categorie_flotteur,phase_journee,maree_port,est_weekend,est_jour_ferie,est_vacances_scolaires,distance_cote_metres,distance_cote_milles_nautiques,maree_coefficient,vent_force,mer_force
0,2,0,Somme,Plaisancier francais,Plaisance,apres-midi,Le Treport,True,False,True,-0.543117,-0.543066,1.087642,-0.342751,-0.700528
1,2,0,Nord,Pecheur francais,Peche,nuit,Dunkerque,False,False,True,3.986601,3.986499,-0.691335,1.602936,1.426473
2,2,0,Somme,Autre,Loisir nautique,apres-midi,Le Touquet,True,False,True,-0.543117,-0.543066,-0.938416,0.305811,0.362972
3,2,0,Pas-de-Calais,Clandestin,Plaisance,nuit,Wissant,False,False,False,3.650107,3.650975,-0.493671,-0.342751,0.362972
4,2,0,Pas-de-Calais,Clandestin,Plaisance,nuit,Wissant,False,False,False,3.650107,3.650975,-0.493671,-0.342751,0.362972
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55354,8,0,Gard,Toutes categories,Plaisance,apres-midi,Arcachon Eyrac,False,False,True,-0.517233,-0.519100,0.000489,0.305811,0.362972
55355,8,0,Herault,Toutes categories,Plaisance,nuit,Arcachon Eyrac,False,False,True,-0.154855,-0.154818,0.000489,-0.342751,0.362972
55356,8,0,Herault,Toutes categories,Peche,apres-midi,Vieux-Boucau,True,True,True,0.569900,0.568954,-0.592503,-0.342751,-1.764029
55357,11,0,Aude,Toutes categories,Loisir nautique,apres-midi,Vieux-Boucau,True,False,False,-0.310160,-0.308200,-0.938416,-1.639875,0.362972


In [21]:
#SMOTE


from imblearn.over_sampling import SMOTENC


smo = SMOTENC(categorical_features=[1,2,3,4,5], sampling_strategy=1, k_neighbors=2)
X = df.drop('mort', axis=1)
y = df['mort']



X_res, y_res = smo.fit_resample(X, y)

#On va créer notre smote, c'est à dire créer des individus s'inspirant de la classe 1 = mort qui est sous-representé
#On crée des X et des Y 


In [22]:
df = pd.concat([X_res,y_res],axis=1)
df
#On concatene les 2

df=df.sample(frac=1).reset_index(drop=True)
df

#Je remélange mes données,

,mois,departement,categorie_personne,categorie_flotteur,phase_journee,maree_port,est_weekend,est_jour_ferie,est_vacances_scolaires,distance_cote_metres,distance_cote_milles_nautiques,maree_coefficient,vent_force,mer_force,mort
0,8,Seine-Maritime,Toutes categories,Plaisance,dejeuner,Le Havre,True,False,True,0.311059,0.310122,1.828883,-0.342751,-1.764029,0
1,8,Seine-Maritime,Toutes categories,Plaisance,apres-midi,Fecamp,False,False,True,1.388499,1.387007,-0.508036,0.657242,-0.700528,1
2,4,Morbihan,Plaisancier francais,Loisir nautique,dejeuner,Port-Navalo,True,False,True,-0.310160,-0.308200,0.395818,1.602936,0.362972,0
3,9,Morbihan,Toutes categories,Plaisance,nuit,Saint-Nazaire,False,False,True,-0.543117,-0.543066,0.421095,-0.991313,-0.700528,1
4,6,Finistere,Toutes categories,Plaisance,apres-midi,Brest,False,False,False,-0.262346,-0.263929,-0.086671,-0.342751,-0.700528,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108189,2,Manche,Pecheur francais,Peche,apres-midi,Cherbourg,False,False,False,-0.387812,-0.389684,0.494650,0.305811,0.362972,0
108190,2,Pyrenees-Atlantiques,Plaisancier francais,Plaisance,matinee,Saint-Jean-de-Luz,False,False,False,-0.543117,-0.543066,0.593482,0.305811,2.489974,0
108191,8,Finistere,Toutes categories,Peche,apres-midi,Paimpol,False,False,True,-0.543117,-0.543066,0.346401,0.305811,0.362972,1
108192,10,Cotes-d'Armor,Toutes categories,Plaisance,nuit,Granville,True,False,False,1.546829,1.546809,1.325437,-0.991313,-0.700528,1


In [23]:
#Test remettre en forme 
essai_key = df[key_columns]
essai_flottant = df[flottant]
essay_bool = df[booleen]





In [24]:
tente = pd.DataFrame(scaler.inverse_transform(essai_flottant, copy=None), columns= flottant)
tente

,distance_cote_metres,distance_cote_milles_nautiques,maree_coefficient,vent_force,mer_force
0,3300.000000,1.780000e+00,107.000000,3.000000,1.0
1,7462.558283,4.026698e+00,59.709305,4.541861,2.0
2,900.000000,4.900000e-01,78.000000,6.000000,3.0
3,0.000000,2.220446e-16,78.511533,2.000000,2.0
4,1084.723927,5.823620e-01,68.236196,3.000000,2.0
...,...,...,...,...,...
108189,600.000000,3.200000e-01,80.000000,4.000000,3.0
108190,0.000000,2.220446e-16,82.000000,4.000000,5.0
108191,0.000000,2.220446e-16,77.000000,4.000000,3.0
108192,8074.244114,4.360092e+00,96.812094,2.000000,2.0


In [25]:

ensemble = pd.concat([essai_key,tente,essay_bool],axis=1)
df = ensemble
df



,mois,mort,departement,categorie_personne,categorie_flotteur,phase_journee,maree_port,distance_cote_metres,distance_cote_milles_nautiques,maree_coefficient,vent_force,mer_force,est_weekend,est_jour_ferie,est_vacances_scolaires
0,8,0,Seine-Maritime,Toutes categories,Plaisance,dejeuner,Le Havre,3300.000000,1.780000e+00,107.000000,3.000000,1.0,True,False,True
1,8,1,Seine-Maritime,Toutes categories,Plaisance,apres-midi,Fecamp,7462.558283,4.026698e+00,59.709305,4.541861,2.0,False,False,True
2,4,0,Morbihan,Plaisancier francais,Loisir nautique,dejeuner,Port-Navalo,900.000000,4.900000e-01,78.000000,6.000000,3.0,True,False,True
3,9,1,Morbihan,Toutes categories,Plaisance,nuit,Saint-Nazaire,0.000000,2.220446e-16,78.511533,2.000000,2.0,False,False,True
4,6,1,Finistere,Toutes categories,Plaisance,apres-midi,Brest,1084.723927,5.823620e-01,68.236196,3.000000,2.0,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
108189,2,0,Manche,Pecheur francais,Peche,apres-midi,Cherbourg,600.000000,3.200000e-01,80.000000,4.000000,3.0,False,False,False
108190,2,0,Pyrenees-Atlantiques,Plaisancier francais,Plaisance,matinee,Saint-Jean-de-Luz,0.000000,2.220446e-16,82.000000,4.000000,5.0,False,False,False
108191,8,1,Finistere,Toutes categories,Peche,apres-midi,Paimpol,0.000000,2.220446e-16,77.000000,4.000000,3.0,False,False,True
108192,10,1,Cotes-d'Armor,Toutes categories,Plaisance,nuit,Granville,8074.244114,4.360092e+00,96.812094,2.000000,2.0,True,False,False


In [26]:

#Valeur Target
df['mort'].value_counts()

0    54097
1    54097
Name: mort, dtype: int64